In [100]:
import os
import pandas as pd
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pickle
import json

AttributeError: 'Image' object has no attribute 'shape'

In [58]:
def crop_center_n(img, n):
    h, w = img.size
    midh, midw = h//2, w//2
    size = n // 2
    patch = img.crop((midw-size, midh-size, midw+size+1, midh+size+1))
    return patch
def patch_channel_mean(patch):
    patch = np.array(patch).reshape([-1, 3])
    mv = np.mean(patch, axis=0)
    return mv


In [83]:
path = "../../data/ACNE04/image_face_detection/"
image_name = os.listdir(path)
sc_dict = {}
path_size = [i for i in range(1, 22, 2)]

In [86]:
for name in image_name:
    img = Image.open(path + name)
    img = img.resize((255, 255))
    sc_list = []
    for n in path_size:
        
        patch = crop_center_n(img, n)
        skin_mean = patch_channel_mean(patch)
        max_gap = np.array([255, 255, 255]) - skin_mean
        min_gap = skin_mean
        data_img = []
        for i in range(img.size[0]):
            for j in range(img.size[1]):
                x,y,z = img.getpixel((i, j))
                pixel = np.array((x,y,z))
                pixel = pixel - skin_mean
                if np.sum(pixel) < 0:
                    pixel = np.true_divide(pixel, min_gap)
                else:
                    pixel = np.true_divide(pixel, max_gap)
                data_img.append(pixel)
        data_img = np.mat(data_img)
        km = KMeans(n_clusters=2)
        label = km.fit_predict(data_img)
        sc = silhouette_score(data_img, label)
        sc_list.append((n, sc))
        print(name, n, sc)
    sc_dict[name] = sc_list
    print(sc_list)
    
    

levle0_0.jpg 1 0.5008335918373479
levle0_0.jpg 3 0.4990502791851509
levle0_0.jpg 5 0.49837700278079966
levle0_0.jpg 7 0.49785289999238536
levle0_0.jpg 9 0.49730612706408295
levle0_0.jpg 11 0.4942316468385123
levle0_0.jpg 13 0.49695556348160674
levle0_0.jpg 15 0.4968609528913288
levle0_0.jpg 17 0.49688208624152963
levle0_0.jpg 19 0.496839991705121
levle0_0.jpg 21 0.49676891217473906
[(1, 0.5008335918373479), (3, 0.4990502791851509), (5, 0.49837700278079966), (7, 0.49785289999238536), (9, 0.49730612706408295), (11, 0.4942316468385123), (13, 0.49695556348160674), (15, 0.4968609528913288), (17, 0.49688208624152963), (19, 0.496839991705121), (21, 0.49676891217473906)]
levle0_1.jpg 1 0.5786655436946162
levle0_1.jpg 3 0.579719247860385
levle0_1.jpg 5 0.5791356520150606
levle0_1.jpg 7 0.578539729835128
levle0_1.jpg 9 0.5773505901720702
levle0_1.jpg 11 0.5774284567259661
levle0_1.jpg 13 0.5773767906859969
levle0_1.jpg 15 0.5770739791696111
levle0_1.jpg 17 0.5768468743989826
levle0_1.jpg 19 0.57

KeyboardInterrupt: 

In [103]:
with open("./Silhouette_Coefficient/sc_dict.pkl", "wb") as f:
    pickle.dump(sc_dict, f)
with open("./Silhouette_Coefficient/sc_dict.json", "w") as f:
    json.dump(sc_dict, f, indent=2)